### 1. Install Dependencies

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117 --upgrade

In [ ]:
!pip install langchain einops accelerate transformers bitsandbytes

### 2. Import Dependencies

In [3]:
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import os
import torch

In [4]:
# Check if cuda is available
torch.cuda.is_available()

True

### 3. Build the Pipeline

In [5]:
# Define Model ID
model_id = "tiiuae/falcon-7b-instruct"
# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
# Load Model
model = AutoModelForCausalLM.from_pretrained(model_id, cache_dir='./workspace/',
    torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto", offload_folder="offload")
# Set PT model to inference mode
model.eval()
# Build HF Transformers pipeline
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusFor

In [6]:
# Test out the pipeline
pipeline('who is issac newton?')

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


[{'generated_text': 'who is issac newton?\nIsaac Newton is an English scientist who made significant contributions to physics, mathematics, and astronomy. He is considered one of the most influential scientists of the 17th and 18th centuries.'}]

### 4. Pass it to Langchain

In [7]:
# Setup prompt template
template = PromptTemplate(input_variables=['input'], template='{input}')
# Pass hugging face pipeline to langchain class
llm = HuggingFacePipeline(pipeline=pipeline)
# Build stacked LLM chain i.e. prompt-formatting + LLM
chain = LLMChain(llm=llm, prompt=template)

In [8]:
# Test LLMChain
response = chain.run('who is issac newton?')

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [9]:
response

'\nIsaac Newton (1642 - 1726) is a British scientist and mathematician known for his contributions to physics, mathematics, and the study of optics. He also made significant contributions to the fields of astronomy and engineering during his lifetime. '

### 5. Import Documents

### 6. Put it all together!

### 7. Build Gradio App

In [ ]:
# Install Gradio for the UI component
!pip install gradio

In [14]:
# Import gradio for UI
import gradio as gr

In [15]:
# Create generate function - this will be called when a user runs the gradio app
def generate(prompt):
    # The prompt will get passed to the LLM Chain!
    return chain.run(prompt)
    # And will return responses

In [19]:
# Define a string variable to hold the title of the app
title = '🦜🔗 Falcon-7b-Instruct'
# Define another string variable to hold the description of the app
description = 'This application demonstrates the use of the open-source `Falcon-40b-Instruct` LLM.'

In [22]:
# Build gradio interface, define inputs and outputs...just text in this
gr.Interface(fn=generate, inputs=["text"], outputs=["text"],
             # Pass through title and description
             title=title, description=description,
             # Set theme and launch parameters
             theme='finlaymacklon/boxy_violet').launch(server_port=8089, share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://52609865d1e825fe4b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
